# Assistant-10<br/>Knowledge Retrieval

Notes:
- As of 01/14/2024 there's still a bug with knowledge retrieval.


### Get an OpenAI client

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import oaihelper as framework

## Get an OpenAI client
client = framework.get_openai_client(api_key=framework.api_KEY,
        api_version=framework.api_version,
        azure_endpoint=framework.api_URI,)

### Load the files

In [2]:
## Files
file_benefits = framework.upload_file(client,"../data/contoso-benefits.txt")
file_401k = framework.upload_file(client,"../data/contoso-401k.txt")
file_leave = framework.upload_file(client,"../data/contoso-medical_leave.txt")

Added file:  assistant-KnL2yFgcyrCmkokBQqiGQviQ 1
Added file:  assistant-Z2i2mytb1I2QMdjZIyIf2x3x 2
Added file:  assistant-AOcvZWSifkvospoLP3A0X3mD 3


### Define the tools to use in the Assistants

In [3]:
tools_list = [        
        {
            "type": "function",
            "function": {
                "name": "send_email",
                "description": "Sends an email to a recipient(s).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "to": {
                            "type": "string",
                            "description": "The email(s) the email should be sent to."
                        },
                        "content": {
                            "type": "string",
                            "description": "The content of the email."
                        }
                    },
                    "required": ["to", "content"]
                }
            }
        },
        {"type","retrieval"}
    ]

### Define the HR Assistant

In [4]:
framework.clear_shelves()

hr_assistant = framework.create_assistant(client,
        name="HR Assistant",
        instructions="You HR support chatbot. Use your knowledge base to best respond to the employee queries. Be friendly, professional, and helpful. If you don't know the answer, please say, 'contact HR for more information.'",
        tools=[{"type": "retrieval"}],
        model=framework.gpt_deployment_name,
        file_ids=[file_benefits.id, file_401k.id, file_leave.id],)

Added assistant:  asst_GkELuxgDchd4IfW9Epx35I8o 1


### Define the delegate to handle function calling

In [5]:
def function_calling_delegate(client, run, thread):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "send_email":
            print("Sending email...")
            email_to = arguments['to']
            email_content = arguments['content']
            framework.send_email({'to': email_to,'content':email_content})
            
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": "Email sent"
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process a user message

In [6]:
framework.generate_response(client,hr_assistant,"Can you list some company benefits? Can you provide a summary for requesting medical leave?", "123", "Alex", function_calling_delegate)
framework.generate_response(client,hr_assistant,"Can you provide me a complete list of company benefits benefits?", "123", "Alex", function_calling_delegate)
framework.generate_response(client,hr_assistant,"What is the 401k matching percentage?", "123", "Alex", function_calling_delegate)


Creating new thread for Alex with user_id 123
Added thread:  thread_w9Y3utOBGrCdytg6r5SgRd3I 1
Generated message: Here are some of the company benefits:

- **Health insurance**: The company covers 100% of the premiums for medical, dental, and vision plans for employees and their dependents, along with offering flexible spending accounts, health savings accounts, and wellness programs.
- **Retirement savings**: There's a 50% match on employee's contributions to their 401(k) plans, up to 6% of their salary. Financial education and planning services are also available to help with retirement goals.
- **Paid time off**: Generous policies are in place, which include vacation, sick leave, holidays, parental leave, and sabbatical leave. Employees are encouraged to take this time to recharge and spend with family and friends【11†source】.

Regarding requesting medical leave, the policy indicates the following steps and entitlements:

1. Notify your supervisor and human resources as soon as possi

"The company matches 50% of the employees' contributions to their 401(k) plans, up to 6% of their salary【10†source】."

### Delete all demo objects

In [7]:
framework.cleanup(client)

Deleting:  1  assistants.
AssistantDeleted(id='asst_GkELuxgDchd4IfW9Epx35I8o', deleted=True, object='assistant.deleted')
Deleting:  1  threads.
ThreadDeleted(id='thread_w9Y3utOBGrCdytg6r5SgRd3I', deleted=True, object='thread.deleted')
Deleting:  3  files.
FileDeleted(id='assistant-KnL2yFgcyrCmkokBQqiGQviQ', deleted=True, object='file')
FileDeleted(id='assistant-Z2i2mytb1I2QMdjZIyIf2x3x', deleted=True, object='file')
FileDeleted(id='assistant-AOcvZWSifkvospoLP3A0X3mD', deleted=True, object='file')
